<a href="https://colab.research.google.com/github/merucode/DL/blob/81-colab-keggle_image/02-02_%5BImage-Classification-TL-CNN%5D_Leaf-Diseases-Identification(improvement).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imformation

* Title : [Leaf Diseases Identification](https://www.kaggle.com/c/plant-pathology-2020-fgvc7)
* Type : Image classification
* Evaluation : ROE ACU
* Model : CNN(transfer learning)
* Python version: 3.10.6
* Basic library version
  * torch(torch==2.0.1+cu118)
  * torchvision(torchvision==0.15.2+cu118)
  * sklearn(scikit-learn==1.2.2)
  * cv2(opencv-python==4.7.0.72)
  * albumentations(albumentations==1.2.1): 이미지 변환기
  * numpy(numpy==1.22.4)
  * pandas(pandas==1.5.3)
  * matplotlib(matplotlib==3.7.1)
  * zipfile, random, math, shutil, os.
* Addtional Library version
  * transformers(transformers==4.31.0): 스케줄러 사용을 위해
  * efficientnet_python(efficientnet-python==0.7.1): 사전 학습 모델
* Improvement
  * (Learning) Scheduler, More epochs
  * (Pred) TTA(테스트 단계 데이터 증강)

# STEP 0. Version check and Install Dependency

Step 0-1. Install Dependency

In [ ]:
!pip install transformers
!pip install efficientnet-pytorch

Step 0-2. Version Check

In [ ]:
import sys
import torch
print(f"Python version:{sys.version}")                  # python
print("Torch version:{}".format(torch.__version__))     # torch
print("cuda version: {}".format(torch.version.cuda))    # cuda
print("cudnn version:{}".format(torch.backends.cudnn.version()))    # cudnn

In [ ]:
!pip list

Step 0-3. Download Data

In [ ]:
!export KAGGLE_USERNAME=*** && export KAGGLE_KEY=*** && kaggle competitions download -c plant-pathology-2020-fgvc7

In [ ]:
from zipfile import ZipFile

data_path = '/content/'

with ZipFile(data_path + 'plant-pathology-2020-fgvc7.zip') as zipper:
  zipper.extractall()

# STEP 1. Check Data

Step 1-1. Check data

In [ ]:
import pandas as pd

data_path = '/content/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
submission.head(3)

Step 1-2. Data Visualize

In [ ]:
# 데이터를 타깃값 별로 추출
healthy = train.loc[train['healthy']==1]
multiple_diseases = train.loc[train['multiple_diseases']==1]
rust = train.loc[train['rust']==1]
scab = train.loc[train['scab']==1]

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

mpl.rc('font', size=15)
plt.figure(figsize=(7, 7))

label = ['healthy', 'multiple diseases', 'rust', 'scab']  # 타깃값 레이블
# 타깃값 분포 파이 그래프
plt.pie([len(healthy), len(multiple_diseases), len(rust), len(scab)],
        labels = label,
        autopct='%.1f%%')

Step 1-3. Data Image Visualize

In [ ]:
import matplotlib.gridspec as gridspec
import cv2 # OpenCV 라이브러리

def show_images(img_ids, rows=2, cols=3):
  assert len(img_ids) <= rows * cols # 이미지가 행/열 개수보다 많으면 오류 발생

  plt.figure(figsize=(15, 8)) # 전체 Figure 크기 설정
  grid = gridspec.GridSpec(rows, cols) # 서브플롯 배치

  # 이미지 출력
  for idx, img_id in enumerate(img_ids):
    img_path = f'{data_path}/images/{img_id}.jpg'       # 이미지 파일 경로
    image = cv2.imread(img_path)                        # 이미지 파일 읽기
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      # 이미지 색상 보정
    ax = plt.subplot(grid[idx])
    ax.imshow(image)                                # 이미지 출력

In [ ]:
# 각 타깃값별 image_id(마지막 6개)
num_of_imgs = 6
last_healthy_img_ids = healthy['image_id'][-num_of_imgs:]
last_multiple_diseases_img_ids = multiple_diseases['image_id'][-num_of_imgs:]
last_rust_img_ids = rust['image_id'][-num_of_imgs:]
last_scab_img_ids = scab['image_id'][-num_of_imgs:]

In [ ]:
show_images(last_healthy_img_ids)

In [ ]:
show_images(last_multiple_diseases_img_ids)

In [ ]:
show_images(last_rust_img_ids)

In [ ]:
show_images(last_scab_img_ids)

# STEP 2. Setting for Modeling

Step 2-1. Seed

In [ ]:
import torch
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)                 # 파이썬 난수 생석이 시드 고정
np.random.seed(seed)              # 넘파이 난수 생성기 시드 고정
torch.manual_seed(seed)           # 파이토치 난수 생성기 시드 고정(CPU 사용시)
torch.cuda.manual_seed(seed)      # 파이토치 난수 생성기 시드 고정(GPU 사용시)
torch.cuda.manual_seed_all(seed)  # 파이토치 난수 생성기 시드 고정(멀티 GPU 사용 시)
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False    # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False      # cudnn 사용 해제

Step 2-2.GPU 장비 설정

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

# STEP 3. Dataset

Step 3-1. Load Data

In [ ]:
import pandas as pd

data_path = '/content/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 세트 분리
train, valid = train_test_split(train,
                                test_size=0.1,                 # 9:1 비율로 test 세트 생성
                                stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']], # 훈련 데이터, 검증 데이터 티깃값 비율 유지
                                random_state=50)

print(f"훈련 데이터 개수: {len(train)}")
print(f"검증 데이터 개수: {len(valid)}")
train.head(3)

Step 3-2. Dataset

In [ ]:
import cv2
from torch.utils.data import Dataset
import numpy as np

class ImageDataset(Dataset):
  # 초기화 메서드(생성자)
  def __init__(self, df, img_dir='./', transform=None, is_test=False):
    super().__init__() # 상송받은 Dataset 생성자 호출
    # 전달받은 인수들 저장
    self.df = df
    self.img_dir = img_dir
    self.transform = transform
    self.is_test = is_test

  # 데이터셋 크기 반환 메서드
  def __len__(self):
    return len(self.df)

  # idx 해당하는 데이터 반환 메서드
  def __getitem__(self, idx):
    img_id = self.df.iloc[idx, 0]               # 이미지 ID
    img_path = self.img_dir + img_id + '.jpg'   # 이미지 파일 경로
    image = cv2.imread(img_path)                # 이미지 파일 읽기
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # 이미지 색상 보정

    if self.transform is not None:  # 변화기가 있다면 이미지 변환
      image = self.transform(image=image)['image']    # albumentations 모듈 변환기
      # image = self.transform(image)                 # torchvision transform 모듈 변환기

    # 테스트 데이터면 이미지 데이터만 반환, 그렇치 않으면 타깃값도 반환
    if self.is_test:
      return image          # 테스트용
    else:
      # 타깃값 4개 중 가장 큰 값의 인덱스
      label = np.argmax(self.df.iloc[idx, 1:5])
      return image, label   # 훈련/검증용

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 훈련 데이터용 변환기
transform_train = A.Compose([
    A.Resize(450, 650),                                # 이미지 크기 조절
    A.RandomBrightnessContrast(brightness_limit=0.2,   # 밝기 대비 조절
                               contrast_limit=0.2,
                               p=0.3),
    A.VerticalFlip(p=0.2),                             # 상하 대칭 변환
    A.HorizontalFlip(p=0.5),                           # 좌우 대칭 변환
    A.ShiftScaleRotate(                                # 이동, 스케일링, 회전 변환
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=30,
        p=0.3),
    A.OneOf([A.Emboss(p=1),                            # 양각화, 날카로움, 불러 효과
             A.Sharpen(p=1),
             A.Blur(p=1)], p=0.3),
    A.PiecewiseAffine(p=0.3),                          # 어파인 변환
    A.Normalize(),                                     # 정규화 변환
    ToTensorV2()                                       # 텐서 변환
])

# 검증/테스트 데이터용 변환기
transform_test = A.Compose([
    A.Resize(450, 650),
    A.Normalize(),
    ToTensorV2()
])

In [ ]:
img_dir = '/content/images/'

dataset_train = ImageDataset(df=train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(df=valid, img_dir=img_dir, transform=transform_test)

Step 3-3. Dataloader(multiprocess)

In [ ]:
# 멀티프로세서 사용 시 데이터 로더 시드 고정
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [ ]:
from torch.utils.data import DataLoader # 데이터 로더 클래스

batch_size = 4

loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size,
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=batch_size,
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)

# STEP 4. Module

Step 4-1. Pretrained Model Load

In [ ]:
from efficientnet_pytorch import EfficientNet # EfficientNet 모듈

# 사전 훈련된 'efficientnet-b7' 모델 불러오기
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4)  # num_classes : 최종 출력 갯수

Step 4-1. Pretrained Model Load(output number control other method)

In [ ]:
from efficientnet_pytorch import EfficientNet # EfficientNet 모듈
import torch.nn as nn

# 사전 훈련된 'efficientnet-b7' 모델 불러오기
model = EfficientNet.from_pretrained('efficientnet-b7')

# 사전 모델 마지막 계층 수정(출력값 갯수 수정)
model._fc = nn.Sequential(
    nn.Linear(model._fc.in_features, model._fc.out_features), # 2560 > 1000
    nn.ReLU(),          # 활성화 함수
    nn.Dropout(p=0.5),  # 50% 드롭아웃
    nn.Linear(model._fc.out_features, 4) # 1000 > 4
)

# STEP 5. Learning
### ★Improvement: Scheduler + More Epochs

Step 5-1. Setting

In [ ]:
import tqdm
from torch.optim.adamw import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 손실 함수
criterion = nn.CrossEntropyLoss()
# 옵티마이저
optim = AdamW(model.parameters(), lr=0.00006, weight_decay=0.0001)

Step 5-2. Learning + ★Improvement: Scheduler + More Epochs

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산
from tqdm.notebook import tqdm             # 진행률 표시

epochs = 1 # 39 # 총 에폭

# ★ Scheduler
from transformers import get_cosine_schedule_with_warmup
scheduler = get_cosine_schedule_with_warmup(optim,
                                            num_warmup_steps=len(loader_train)*3,
                                            num_training_steps=len(loader_train)*epochs)

for epoch in range(epochs):
  # == [훈련] ================================
  model.train()         # 모델을 훈련 상태로 설정
  epoch_train_loss = 0  # 에폭별 손실값 초기화(훈련 데이터용)

  # 반복 횟수 만큼 반복
  for images, labels in tqdm(loader_train):
    # 이미지, 레이블 데이터 미니배치를 장비에 할당
    images = images.to(device)
    labels = labels.to(device)

    optim.zero_grad()         # 옵티마이저 기울기 초기화
    outputs = model(images)   # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
    loss = criterion(outputs, labels)   # 손실값 계산(예측값과 타깃값의 오차)
    epoch_train_loss += loss.item() # 현재 배치에서의 손실 추가
    loss.backward()           # 역전파 수행
    optim.step()     # 가중치 갱신
    scheduler.step()

  # 훈련 데이터 손실값 출력
  print(f"에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_train_loss/len(loader_train):.4f}")

  # == [검증] ================================
  model.eval()          # 모델을 평가 상태로 설정
  epoch_valid_loss = 0  # 에폭별 손실값 초기화(검증 데이터용)
  preds_list = []       # 예측 확률값 저장용 리스트 초기화
  true_onehot_list = [] # 실제 타깃값 저장용 리스트 초기화

  with torch.no_grad(): # 기울기 계산 비활성화
    # 미니 배치 단위로 검증
    for images, labels in loader_valid:
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)                #  순전파
      loss = criterion(outputs, labels)      # 손실값 계산(검증 데이터용)
      epoch_valid_loss += loss.item()

      # 예측값 및 실제값 계산
      preds = torch.softmax(outputs.cpu(), dim=1).numpy()  # 예측 확률 값
      labels = labels.to("cpu") # ERROR HANDLING: indices should be either on cpu or on the same device as the indexed tensor (cpu)
      true_onehot = torch.eye(4)[labels].cpu().numpy()     # 실제값 (원-핫 인코딩 형식)
      # 예측 확률값과 실제값 저장
      preds_list.extend(preds)
      true_onehot_list.extend(true_onehot)

  # 검증 데이터 손실값 및 ROC AUC 점수 출력
  print(f"에폭 [{epoch+1}/{epochs}] - 검증 데이터 손실값: {epoch_valid_loss/len(loader_valid):.4f} / \
  검증 데이터 ROC AUC: {roc_auc_score(true_onehot_list, preds_list):.4f}")

# STEP 6. Validation(STEP 5 동시 진행)

Step 6-1. Setting

Step 6-2. Model Validation

# STEP 7. Evaluation and Submission
### ★Improvement: TTA(테스트 단계 데이터 증강)

Step 7-1. Setting + ★Improvement: TTA

In [ ]:
dataset_test = ImageDataset(df=submission, img_dir=img_dir,
                            transform=transform_test, is_test=True)
loader_test = DataLoader(dataset=dataset_test, batch_size=batch_size,
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)

# ★ TTA용 데이터 셋 및 데이터 로더
dataset_TTA = ImageDataset(df=submission, img_dir=img_dir,
                            transform=transform_train, is_test=True)
loader_TTA = DataLoader(dataset=dataset_test, batch_size=batch_size,
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)


Step 7-2. Evaluation

In [ ]:
model.eval()  # 모델 평가 상태로 설정

# 원본 데이터로 예측
preds_test = np.zeros((len(test), 4)) # 예측값 저장용 배열 초기화

with torch.no_grad(): # 기울기 계산 비활성화
  for i, images in enumerate(loader_test):
    # 이미지 데이터 미니배치를 장비에 할당
    images = images.to(device)

    # 순전파
    outputs = model(images)
    # 타깃값 예측 확률
    preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
    # preds에 preds_part 이어 붙이기
    preds_test[i*batch_size:(i+1)*batch_size] += preds_part

In [ ]:
submission_test = submission.copy() # 제출 샘플 파일 복사
submission_test[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_test   # submission df 결과값 재설정

Step 7-3.★TTA

In [ ]:
num_TTA = 7

# TTA 데이터로 예측
preds_tta = np.zeros((len(test), 4)) # 예측값 저장용 배열 초기화(TTA)

# TTA 적용 예측
for i in range(num_TTA):
  with torch.no_grad(): # 기울기 계산 비활성화
    for i, images in enumerate(loader_test):
      # 이미지 데이터 미니배치를 장비에 할당
      images = images.to(device)

      # 순전파
      outputs = model(images)
      # 타깃값 예측 확률
      preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
      # preds에 preds_part 이어 붙이기
      preds_tta[i*batch_size:(i+1)*batch_size] += preds_part

preds_tta /= num_TTA # 누적값의 평균

In [ ]:
submission_tta = submission.copy() # 제출 샘플 파일 복사
submission_tta[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_tta   # submission df 결과값 재설정

Step 7-4. ★Label Smoothing

In [ ]:
def apply_label_smoothiing(df, target, alpha, threshold):
  df_target = df[target].copy() # 타깃값 복사
  k = len(target)               # 타깃값 개수

  for idx, row in df_target.iterrows(): #
    if (row > threshold).any():         # 각 타깃값에 대해 임계값 넘는지 확인(임계값 넘으면 과잉 확신)
      row = (1 - alpha)*row + alpha/k   # 레이블 스무딩 적용
      df_target.iloc[idx] = row
  return df_target

In [ ]:
alpha = 0.001     # 레이블 스무딩 강도
threshold = 0.999 # 레이블 스무딩을 적용할 임계값

# 레이블 스무딩을 위해 df 복사
submission_test_ls = submission_test.copy()
submission_tta_ls = submission_tta.copy()

target = ['healthy', 'multiple_diseases', 'rust', 'scab'] # 타깃값 열 이름

# 레이블 스무딩 적용
submission_test_ls[target] = apply_label_smoothiing(submission_test_ls, target,
                                                    alpha, threshold)
submission_tta_ls[target] = apply_label_smoothiing(submission_tta_ls, target,
                                                    alpha, threshold)

Step 7-5.Submission

In [ ]:
submission_test.to_csv('submission_test.csv', index=False)      # 제출 파일 생성
submission_tta.to_csv('submission_tta.csv', index=False)        # 제출 파일 생성
submission_test_ls.to_csv('submission_test_ls.csv', index=False)        # 제출 파일 생성
submission_tta_ls.to_csv('submission_tta_ls.csv', index=False)        # 제출 파일 생성